<a href="https://colab.research.google.com/github/Gautam12NI/102303739_UML_501_assignments/blob/main/102303739_Ass3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:


import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

def least_square_beta(X, y):
    X = np.insert(X, 0, 1, axis=1)
    beta = np.linalg.inv(X.T @ X) @ X.T @ y
    return beta

df = pd.read_csv('/content/USA_Housing.csv')

X = df.drop('Price', axis=1).values
y = df['Price'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

r2_scores = []
betas = []
for train_index, test_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]
    beta = least_square_beta(X_train, y_train)
    betas.append(beta)
    y_pred = np.insert(X_test, 0, 1, axis=1) @ beta
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)
    print(f"R2 Score: {r2}")

best_beta = betas[np.argmax(r2_scores)]
print(f"\nBest Beta: {best_beta}")

X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
y_pred_final = np.insert(X_test_final, 0, 1, axis=1) @ best_beta
final_r2 = r2_score(y_test_final, y_pred_final)
print(f"Final R2 Score on 30% test data with best beta: {final_r2}")

R2 Score: 0.9179971706985147
R2 Score: 0.9145677884802818
R2 Score: 0.9116116385364478
R2 Score: 0.9193091764960816
R2 Score: 0.9243869413350316

Best Beta: [1.23161736e+06 2.30225051e+05 1.63956839e+05 1.21115120e+05
 7.83467170e+02 1.50662447e+05]
Final R2 Score on 30% test data with best beta: 0.9147458156636434


In [12]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_csv("USA_Housing.csv")  # change filename if needed

# Input features and output
X = df.drop("Price", axis=1).values
y = df["Price"].values.reshape(-1, 1)  # make y a column vector

# Scale features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split into Train (56%), Validation (14%), Test (30%)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)   # 0.2 * 0.7 ≈ 0.14

# Add bias term
def add_bias(X):
    return np.c_[np.ones((X.shape[0], 1)), X]

X_train_b = add_bias(X_train)
X_val_b = add_bias(X_val)
X_test_b = add_bias(X_test)

# Gradient Descent function
def gradient_descent(X, y, lr, iterations):
    n_samples, n_features = X.shape
    beta = np.zeros((n_features, 1))  # initialize coefficients
    for i in range(iterations):
        y_pred = X @ beta
        error = y_pred - y
        gradient = (2/n_samples) * (X.T @ error)
        beta -= lr * gradient
    return beta

learning_rates = [0.001, 0.01, 0.1, 1]
results = []

for lr in learning_rates:
    beta = gradient_descent(X_train_b, y_train, lr, 1000)

    y_val_pred = X_val_b @ beta
    y_test_pred = X_test_b @ beta

    r2_val = r2_score(y_val, y_val_pred)
    r2_test = r2_score(y_test, y_test_pred)

    results.append({
        "learning_rate": lr,
        "beta": beta,
        "r2_val": r2_val,
        "r2_test": r2_test
    })
    print(f"Learning Rate: {lr}")
    print(f"Validation R2: {r2_val:.4f}, Test R2: {r2_test:.4f}\n")

# Find best model based on validation R2
best_model = max(results, key=lambda x: x["r2_val"])
print("Best Model based on Validation R2")
print("Learning Rate:", best_model["learning_rate"])
print("Validation R2:", best_model["r2_val"])
print("Test R2:", best_model["r2_test"])
print("Beta Coefficients:\n", best_model["beta"])

Learning Rate: 0.001
Validation R2: 0.6820, Test R2: 0.6490

Learning Rate: 0.01
Validation R2: 0.9098, Test R2: 0.9148

Learning Rate: 0.1
Validation R2: 0.9098, Test R2: 0.9148

Learning Rate: 1
Validation R2: -inf, Test R2: -inf

Best Model based on Validation R2
Learning Rate: 0.01
Validation R2: 0.909799626728122
Test R2: 0.9147569598865972
Beta Coefficients:
 [[1232618.31836202]
 [ 230067.95333238]
 [ 163710.26584918]
 [ 121680.22876975]
 [   2833.37135223]
 [ 150657.57448494]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_regression.py:1275: RuntimeWarning: overflow encountered in square
  numerator = xp.sum(weight * (y_true - y_pred) ** 2, axis=0)
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_regression.py:1275: RuntimeWarning: overflow encountered in square
  numerator = xp.sum(weight * (y_true - y_pred) ** 2, axis=0)


In [11]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA

# 1. Load the dataset with specified column names and replace '?' with NaN
column_names = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration",
                "num_doors", "body_style", "drive_wheels", "engine_location",
                "wheel_base", "length", "width", "height", "curb_weight",
                "engine_type", "num_cylinders", "engine_size", "fuel_system",
                "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm",
                "city_mpg", "highway_mpg", "price"]
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data',
                 names=column_names, na_values='?')

# 2. Imputation and dropping rows
for col in ['normalized_losses', 'horsepower', 'peak_rpm', 'price', 'bore', 'stroke']:
    imputer = SimpleImputer(strategy='mean')
    df[col] = imputer.fit_transform(df[[col]])
df.dropna(subset=['price'], inplace=True) # Drop rows with NaN in price column

# 3. Non-numeric to numeric conversion
# (i) 'num_doors' and 'num_cylinders'
door_map = {'two': 2, 'four': 4}
df['num_doors'] = df['num_doors'].map(door_map).fillna(4)
cylinder_map = {'two': 2, 'three': 3, 'four': 4, 'five': 5, 'six': 6, 'eight': 8, 'twelve': 12}
df['num_cylinders'] = df['num_cylinders'].map(cylinder_map)

# (ii) Dummy encoding for 'body_style' and 'drive_wheels'
df = pd.get_dummies(df, columns=['body_style', 'drive_wheels'], drop_first=True)

# (iii) Label encoding
label_encoder = LabelEncoder()
for col in ['make', 'aspiration', 'engine_location', 'fuel_type']:
    df[col] = label_encoder.fit_transform(df[col])

# (iv) 'fuel_system' encoding
df['fuel_system'] = df['fuel_system'].apply(lambda x: 1 if 'pfi' in str(x) else 0)

# (v) 'engine_type' encoding
df['engine_type'] = df['engine_type'].apply(lambda x: 1 if 'ohc' in str(x) else 0)

# 4. Divide and scale
X = df.drop('price', axis=1)
y = df['price']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 5. Train and test without PCA
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)
y_pred = linear_reg.predict(X_test)
r2_before_pca = r2_score(y_test, y_pred)
print(f"R2 Score before PCA: {r2_before_pca}")

# 6. Reduce dimensionality with PCA and retrain
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y, test_size=0.3, random_state=42)

linear_reg_pca = LinearRegression()
linear_reg_pca.fit(X_train_pca, y_train_pca)
y_pred_pca = linear_reg_pca.predict(X_test_pca)
r2_after_pca = r2_score(y_test_pca, y_pred_pca)
print(f"R2 Score after PCA: {r2_after_pca}")

if r2_after_pca > r2_before_pca:
    print("\nPerformance improved after PCA.")
elif r2_after_pca < r2_before_pca:
    print("\nPerformance decreased after PCA.")
else:
    print("\nPerformance remained the same after PCA.")

R2 Score before PCA: 0.804442243576259
R2 Score after PCA: 0.7500675882701553

Performance decreased after PCA.
